In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt

In [3]:
covid_kr = pd.read_csv('../data/covid_korea/Covid19GenAgeCaseInf.csv')

In [4]:
covid_kr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10673 entries, 0 to 10672
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   seq           10673 non-null  int64  
 1   gubun         10673 non-null  object 
 2   confCase      10673 non-null  int64  
 3   confCaseRate  10673 non-null  float64
 4   death         9184 non-null   float64
 5   deathRate     10610 non-null  float64
 6   criticalRate  9007 non-null   object 
 7   createDt      10673 non-null  object 
 8   updateDt      4 non-null      object 
dtypes: float64(3), int64(2), object(4)
memory usage: 750.6+ KB


In [5]:
16054424 +13873534

29927958

In [6]:

16855.0+16249.0

33104.0

In [7]:
(33104.0 / 29927958) * 100

0.1106122910223277